In [2]:
import pandas as pd

In [31]:
stocks = pd.read_json('https://financialmodelingprep.com/api/v3/company/stock/list')['symbolsList']

In [32]:
stocks[0]

{'symbol': 'SPY',
 'name': 'SPDR S&P 500',
 'price': 294.94,
 'exchange': 'NYSE Arca'}

In [54]:
stocks

0        {'symbol': 'SPY', 'name': 'SPDR S&P 500', 'pri...
1        {'symbol': 'CMCSA', 'name': 'Comcast Corporati...
2        {'symbol': 'KMI', 'name': 'Kinder Morgan Inc.'...
3        {'symbol': 'INTC', 'name': 'Intel Corporation'...
4        {'symbol': 'MU', 'name': 'Micron Technology In...
                               ...                        
14120    {'symbol': 'ESCR', 'name': 'Xtrackers Bloomber...
14121    {'symbol': 'ESHY', 'name': 'Xtrackers J.P. Mor...
14122    {'symbol': 'SQEW', 'name': 'LeaderShares Equit...
14123    {'symbol': 'GNRSW', 'name': 'Greenrose Acquisi...
14124    {'symbol': 'MSVX', 'name': 'ETF Series Solutio...
Name: symbolsList, Length: 14125, dtype: object

In [53]:
for stock in stocks:
    url_metrics = 'https://financialmodelingprep.com/api/v3/company-key-metrics/{}?period=quarter'.format(stock['symbol'])
    df_metrics = pd.read_json(url_metrics)
    if len(df_metrics) == 0:
        stock['metrics'] = []
    else:
        stock['metrics'] = df_metrics['metrics']

KeyboardInterrupt: 

""


In [52]:
len(df_metrics)

0